### Load Data  

Load all the negative review comments for property # 218524

In [1]:
fileName="/Users/raghuramsrinivas/Documents/education/Stats/Backup-Pyscripts/Immersion/badReviews_218524.csv"

In [2]:
import pandas as pd
df=pd.read_csv(fileName)
df["ReviewKey"]=df["Unnamed: 0"]

In [3]:
df.ReviewKey.astype("int64",inplace=True)
df.head(3)

,Unnamed: 0,Author,BusinessService,CheckIn,Cleanliness,Content,File,Location,Overall,Rooms,Service,Value,Overall1,ReviewKey
0,2,cleanwatergirl,-1,5,3,too sick to enjoy The ride to and form the air...,/Users/raghuramsrinivas/Downloads/Review_Texts...,2,2,2,2,1,2,2
1,5,CR_ONT,3,1,5,Beautiful resort but be careful of a few thing...,/Users/raghuramsrinivas/Downloads/Review_Texts...,5,3,5,4,4,3,5
2,7,marksdallastx,2,1,2,Definitely not a 5 star resort I'm dumbfounded...,/Users/raghuramsrinivas/Downloads/Review_Texts...,4,2,1,2,1,2,7


### Initialize Watson Developer Cloud API objects

In [4]:
import json
from os.path import join, dirname
from watson_developer_cloud import AlchemyLanguageV1

In [5]:
from watson_developer_cloud import ConceptInsightsV2
concept_insights = ConceptInsightsV2(username='c70d04a9-779a-4862-b36a-0289912e173a',password='FTQModH127el')

In [6]:
strSampleText="How Watson answers a question and for not so simple steps.A first person mentioned by name in the man in the iron mask is this hero of previous book by the same author.This is a typical jeopardy clue presented in the tricky Jeopardy game show format that can be pretty difficult for a person to understand much less thecomputer.A computer understands the text using the numbers ones and zeros.Watson can see and hear so audio and visual questions are within limits."

print(strSampleText)

How Watson answers a question and for not so simple steps.A first person mentioned by name in the man in the iron mask is this hero of previous book by the same author.This is a typical jeopardy clue presented in the tricky Jeopardy game show format that can be pretty difficult for a person to understand much less thecomputer.A computer understands the text using the numbers ones and zeros.Watson can see and hear so audio and visual questions are within limits.


In [7]:
concept_insights.annotate_text(strSampleText,graph='/graphs/wikipedia/en-latest')

{u'annotations': [{u'concept': {u'id': u'/graphs/wikipedia/en-20120601/concepts/Watson_(computer)',
    u'label': u'Watson (computer)'},
   u'score': 0.7763709,
   u'text_index': [4, 10]},
  {u'concept': {u'id': u'/graphs/wikipedia/en-20120601/concepts/Question',
    u'label': u'Question'},
   u'score': 0.76485574,
   u'text_index': [437, 446]},
  {u'concept': {u'id': u'/graphs/wikipedia/en-20120601/concepts/Visual_perception',
    u'label': u'Visual perception'},
   u'score': 0.687569,
   u'text_index': [430, 436]},
  {u'concept': {u'id': u'/graphs/wikipedia/en-20120601/concepts/Computer',
    u'label': u'Computer'},
   u'score': 0.6854849,
   u'text_index': [330, 338]},
  {u'concept': {u'id': u'/graphs/wikipedia/en-20120601/concepts/Man_in_the_Iron_Mask',
    u'label': u'Man in the Iron Mask'},
   u'score': 0.68048066,
   u'text_index': [98, 118]},
  {u'concept': {u'id': u'/graphs/wikipedia/en-20120601/concepts/Hero',
    u'label': u'Hero'},
   u'score': 0.66466016,
   u'text_index':

### Annotate all negative review comments using ConceptsInsights.annotate_text()

In [8]:
import sys  
#reload(sys)  
#sys.setdefaultencoding('utf8')

In [ ]:
## Warning : Takes a long time to run!
concept_counts ={}
label_counts={}
i=0
listConcepts = []
for index,row in df.iterrows():
    try:
        
        
        strTextContent = row["Content"]
        strReviewKey=row["ReviewKey"]
        strTextContent=strTextContent.encode("utf8")
        docConcepts = concept_insights.annotate_text(strTextContent,graph='/graphs/wikipedia/en-latest')
        
        docConceptsArray =docConcepts["annotations"]

        for j in range(0,len(docConceptsArray)):
            dfDict={}
            dict1 = docConceptsArray[j]
            dfDict["Concept"] = dict1["concept"]["id"]
            dfDict["Label"] = dict1["concept"]["label"]
            dfDict["Score"]=dict1["score"]
            dfDict["ReviewKey"]=strReviewKey
            
            listConcepts.append(dfDict)
            i=i+1
            #if(i==3):
            #    break
    except Exception , e:
        print("Exception occured at "+str(i))
        print(e)
    
dfConcepts=pd.DataFrame(listConcepts)       
dfConcepts.to_csv("ConceptExtraction_badReviews.csv")        
dfConceptsGrpBy = dfConcepts.groupby([dfConcepts.ReviewKey,dfConcepts.Label]).mean()
dfConceptsGrpBy.to_csv("ConceptExtraction_badReviews_grouped.csv")

### Analyze the top concepts mentioned in the negative review comments

In [17]:
dfConceptsExtractions = pd.read_csv("ConceptExtraction_badReviews_grouped.csv")

##### List all the concepts mentioned in each review comment

In [9]:
dfConceptsExtractions.head(5)

,ReviewKey,Label,Score
0,2,Airport,0.261225
1,2,Diarrhea,0.573177
2,2,Dominican Republic,0.287575
3,2,Food,0.604185
4,2,Health,0.506297


##### Get the # of occurance of each concept label

In [18]:
dfConceptsLabelCounts = dfConceptsExtractions.groupby(dfConceptsExtractions.Label).count()
dfConceptsLabelCounts.to_csv("dfConceptsLabelCounts.csv")
dfConceptsLabelCounts=pd.read_csv("dfConceptsLabelCounts.csv")

##### Extract the most frequently occuring concepts across all negative reviews

In [19]:
dfConceptsLabelCounts=dfConceptsLabelCounts[dfConceptsLabelCounts.Score>7]

In [20]:
import plotly.plotly as py
import plotly.graph_objs as go

py.sign_in('rags_555', 'lgf41szz6a')
trace0 = go.Bar(
    x=dfConceptsLabelCounts.Label,
    y=dfConceptsLabelCounts.Score,
    opacity=0.6
)

data = [trace0]
layout = go.Layout(
    title='Concepts Counts',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='ConceptsCounts')

In [21]:
def getRelatedText(strConcept):
    relatedReviewKeys = dfConceptsExtractions[dfConceptsExtractions.Label==strConcept]
    listReviewKeys = relatedReviewKeys.ReviewKey.unique()
    dfFilteredReview = df[df.ReviewKey.isin(listReviewKeys)]
    i =0
    for index,row in dfFilteredReview.iterrows():
        print(row["Content"])
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n\n")
        if (i==5):
            break
        i=i+1

In [25]:
getRelatedText("Airport")



too sick to enjoy The ride to and form the airport is not good. Very bumpy road and you can really see the poor living conditions. When you get to the resort it is like another world. we were told not to drink the tap or shower water- keep your mouth shut in the shower etc. but we have since heard that not all properties in punta cana have a water problem. In some resorts they have a huge purification system and the tap water is safe. Both my husband and I were sick with diarrhea while we were there and put it down to the unclean water used to prepare food. We will not go on any more holidays where we have to worry about our health. Each night at about 10PM when we were settling down to sleep the music started up and we like fresh air but we had to close our patio doors to mute the sound. We really wanted a quieter resort. It was disappointing. It was partially our fault for not doing our research properly. Some people did not hear the bands play at night but we couldnt change our swim

In [26]:
getRelatedText("Air conditioning")

Facilities need work!!! We visited Excellence for 5 nights in December. Our first room, #1112, had a safe that didn't work and so-so air conditioning. When we went to the front desk to complain, we were told to go to the room and someone would be there within 15 minutes. 45 minutes later, the safe guy showed up, but nobody for the A/C. The safe guy couldn't fix it. When he left, the electricity went out! It went out a second time before we finally went to the front desk to change rooms. We had dinner that night in the Lobster House. Don't waste your time on this one...the lobster tails had about 2 bites of food included. While we were in there, the electricity went out again!Room 3002 served us pretty well, until night #3 when my partner got up to go to the bathroom and stepped into an inch of water. A hose had broken on the back of the toilet and flooded our room. It would've been ok, but when we went to the front desk we were told that we needed to wait until noon to see if perhaps t

In [27]:
getRelatedText("Room service")


NEVER EVER AGAIN, don't go!! Fist of all we are not naive, are extremely well travelled and are realistic in our expectations!!! We ignored most of the negative comments from trip advisor and took the common notion of people just like to complain (we didn't expect to be doing the same 14 days later!!) so we departed with our hard earned cash and booked this amazing holiday, a romantic get a way, adult only, with expectations of relaxation, good food, friendly service and a stress free holiday!! we understand that it's the Caribbean, that it's a tired resort and are the first to say that the grounds on which the resort is are breathe taking, but they shouldn't be graded as a 5 star hotel and charging you for the privileged.ACCOMMODATION:So we arrived at the hotel, late unfortunately we had had a delay in flight not the hotels or anyone's fault just one of them things!! but everything was going to be great for the next 14 days, well not quite, we had made sure when booking that we had a 